In [11]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [12]:
!pip install hdbscan

In [13]:
###############################################################################
# NOTE : L'algorithme va ajouter une colonne 'cluster_label' dans le DataFrame
#        et donc dans le CSV final. Les valeurs correspondront aux numéros
#        de cluster identifiés. Si l'étiquette vaut -1, alors le point sera
#        considéré comme du « bruit » (noise) et n'appartiendra à aucun cluster.
###############################################################################

In [14]:
###############################################################################
#                          BLOC DE CONFIGURATION                              #
###############################################################################

import os
import pandas as pd
import numpy as np

# Bibliothèque HDBSCAN
# !pip install hdbscan
import hdbscan

from typing import List
from sklearn.preprocessing import MinMaxScaler


###############################################################################
# Paramètres de chargement de données
###############################################################################

# Chemin du dossier dans lequel se trouve le CSV
folder_path = "/content/drive/MyDrive/COLAB_NLP/DATA/"

# Nom du fichier CSV à traiter
csv_file = "transcripts.csv"

# Séparateur par défaut (utilisé en fallback) : ',' ou ';'
# La détection automatique ajustera cette valeur.
default_sep = ","


###############################################################################
# Paramètres de clustering
###############################################################################

# Colonnes à utiliser pour le clustering (elles doivent être numériques !)
columns_to_use = ["video_view_count", "video_like_count", "video_comment_count", "sentiment"]

# Taille minimale d'un cluster.
# Il est crucial de définir une valeur élevée pour `min_cluster_size` (par exemple, 1000)
# afin d'éviter la formation de micro-clusters difficiles, voire impossibles, à interpréter.
# Une valeur élevée garantit que les clusters sont plus larges, plus "stables", et reste dans un
# scope raisonnable pour une analyse humaine (idéalement, entre 10/15 clusters maximum).
# Si vous diminuez cette valeur, vous autorisez la formation de clusters plus petits,
# qui peuvent révéler des sous-groupes fins, mais risquent de manquer de signification
# ou de complexifier inutilement l'interprétation.
min_cluster_size = 1000

# Méthode de sélection finale des clusters.
# - 'leaf' : on découpe la hiérarchie jusqu’aux feuilles stables.
# - 'eom' : on fait un trade-off en remontant la hiérarchie pour fusionner certaines feuilles
#           jugées insuffisamment stables individuellement.
cluster_selection_method = 'eom'

"\nMéthode de sélection finale des clusters.\n'leaf' = on découpe la hiérarchie jusqu’aux feuilles stables\n'eom' = on fait un trade off en remontant la hiérarchie pour fusionner certaines feuilles \n        jugées insuffisamment stables individuellement.\n"

In [15]:
def detect_separator(csv_file_path: str) -> str:
    """
    Détecte s’il y a plus de virgules ou de points-virgules dans la première ligne
    du fichier pour faire un guess sur le délimiteur (',' ou ';').
    """
    with open(csv_file_path, 'r', encoding='utf-8') as f:
        first_line = f.readline()
    nb_commas = first_line.count(',')
    nb_semicolons = first_line.count(';')
    return ';' if nb_semicolons > nb_commas else ','

In [16]:
def load_data(csv_file_path: str, user_default_sep: str = ',') -> pd.DataFrame:
    """
    1. Détecte le séparateur via detect_separator().
    2. Charge les données avec pandas, en se basant sur le séparateur détecté.
    """
    sep_found = detect_separator(csv_file_path)
    print(f"[*] Séparateur détecté : '{sep_found}' (défaut : '{user_default_sep}')")

    df = pd.read_csv(csv_file_path, sep=sep_found, encoding='utf-8')
    print(f"[*] CSV chargé : {df.shape[0]} lignes, {df.shape[1]} colonnes.")
    return df

In [17]:
def apply_hdbscan(df: pd.DataFrame,
                  features: List[str],
                  min_cluster_size: int = 5,
                  cluster_selection_method: str = 'eom'
                  ) -> np.ndarray:
    """
    Applique l'algorithme de clustering HDBSCAN sur les colonnes spécifiées (features).
    Retourne un tableau numpy des labels de cluster pour chaque ligne du DataFrame.

    Paramètres :
      - df : DataFrame initial
      - features : liste de colonnes numériques à utiliser pour le clustering
      - min_cluster_size : taille minimale d'un cluster
      - cluster_selection_method : méthode de sélection de clusters ('eom' ou 'leaf')

    Retour :
      - cluster_labels : un array contenant le numéro de cluster pour chaque point
                         (ou -1 si le point est considéré comme du bruit).
    """

    # Vérification de l'existence des colonnes
    for col in features:
        if col not in df.columns:
            raise ValueError(f"La colonne '{col}' est introuvable dans le DataFrame.")

    # Extraire les données numériques
    data_for_clustering = df[features].values

    # --------------------------------------------------------------------------
    # Application du MinMaxScaler pour ramener chaque feature dans [0, 1].
    # Cela évite qu'une colonne à grande échelle ne domine le calcul des distances.
    # --------------------------------------------------------------------------
    scaler = MinMaxScaler()
    data_for_clustering_scaled = scaler.fit_transform(data_for_clustering)

    # Initialiser le clusterer HDBSCAN
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=None,
        metric='euclidean',
        cluster_selection_method=cluster_selection_method
    )

    # Ajuster sur les données (maintenant mises à l'échelle)
    print("[*] Démarrage du clustering HDBSCAN...")
    clusterer.fit(data_for_clustering_scaled)
    print("[*] Clustering HDBSCAN terminé.")

    # clusterer.labels_ : tableau contenant le label du cluster de chaque point
    cluster_labels = clusterer.labels_

    return cluster_labels

In [18]:
def save_clusters_to_csv(df: pd.DataFrame,
                         cluster_labels: np.ndarray,
                         features: list,
                         overwrite: bool = False,
                         output_path: str = "hdbscan_output.csv",
                         sep: str = ','):
    """
    1. Crée un nom de colonne qui inclut tous les noms de variables utilisées,
       séparés par '_'.
    2. Ajoute les labels de cluster au DataFrame.
    3. Sauvegarde le DataFrame mis à jour au format CSV.

    - df             : DataFrame initial
    - cluster_labels : array des labels de cluster (même longueur que df)
    - features       : liste de colonnes utilisées
    - overwrite      : True si on veut écraser le CSV d'origine, False sinon
    - output_path    : chemin du CSV de sortie
    - sep            : séparateur CSV
    """

    # Créer la colonne de cluster, par ex. "cluster_var1_var2_var3"
    cluster_col_name = "cluster_" + "_".join(features)

    # Ajouter au DataFrame
    df[cluster_col_name] = cluster_labels

    # Export CSV
    df.to_csv(output_path, index=False, sep=sep, encoding='utf-8')
    print(f"[*] Fichier CSV enregistré dans : {output_path}")
    print(f"    -> Nouvelle colonne : {cluster_col_name}")

In [19]:
def main():
    # 1) Construit le chemin complet vers le CSV
    full_csv_path = os.path.join(folder_path, csv_file)

    # 2) Charge le CSV
    df = load_data(full_csv_path, user_default_sep=default_sep)

    # 3) Applique HDBSCAN sur les features sélectionnées
    cluster_labels = apply_hdbscan(
        df,
        features=columns_to_use,
        min_cluster_size=min_cluster_size,
        cluster_selection_method=cluster_selection_method
    )

    # 5) Sauvegarde le résultat dans un CSV
    #    On réutilise le même séparateur détecté au chargement ?
    #    Pour cela, on redétecte ou on se fie à default_sep
    #    (selon vos préférences, vous pouvez aussi enregistrer avec un nouveau séparateur).
    real_sep = detect_separator(full_csv_path)
    save_clusters_to_csv(
        df=df,
        cluster_labels=cluster_labels,
        features=columns_to_use,
        overwrite=True,
        output_path=full_csv_path,
        sep=real_sep
    )

    print("[*] Script HDBSCAN terminé avec succès.")

In [20]:
if __name__ == "__main__":
    main()

[*] Séparateur détecté : ';' (défaut : ',')
[*] CSV chargé : 26108 lignes, 15 colonnes.
[*] Démarrage du clustering HDBSCAN...


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[*] Clustering HDBSCAN terminé.
[*] Fichier CSV enregistré dans : /content/drive/MyDrive/COLAB_NLP/DATA/transcripts.csv
    -> Nouvelle colonne : cluster_video_view_count_video_like_count_video_comment_count_sentiment
[*] Script HDBSCAN terminé avec succès.
